<a href="https://colab.research.google.com/github/hyxxnii/Tave-6th-Project/blob/master/NIPA%20%EB%B3%B8%EC%84%A0%20-%20%EC%9C%A1%EA%B5%B0%20%EB%AF%BC%EC%9B%90%20%EB%B6%84%EB%A5%98%20%EB%AA%A8%EB%8D%B8_try3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

train_path = '../data/.train/.task145/data/train.tsv'
test_path = '../data/.train/.task145/data/test.tsv'

In [ ]:
train_df = pd.read_csv(train_path, sep='\t')
train_df.head()

In [ ]:
def read_documents(filename):
    with open(filename, encoding="utf-8") as f: # 윈도우는 꼭 encoding 해줘야 함
        documents = [line.split('\t') for line in f.read().splitlines()] 
        documents = documents[1:] # 첫번째 줄이 카테고리 이름적혀있는 줄이라서 날려버림
        
    return documents

test = read_documents(test_path)
test_df = pd.DataFrame(test)
test_df.columns = ["comment", "tag"]
test_df.head()

In [ ]:
train_org = train_df.copy()
test_org = test_df.copy()

In [ ]:
train_df['tag'].value_counts()
# 불균형

In [ ]:
import konlpy 
from konlpy.tag import Mecab, Kkma, Okt, Komoran
import json
import os
import re
from pprint import pprint

def text_cleaning(doc):
    # 한국어를 제외한 글자를 제거하는 함수.
    doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    return doc

def define_stopwords(path):
    SW = set()
    with open(path, encoding='utf-8') as f:
        for word in f:
            SW.add(word)
            
    return SW

def text_tokenizing(doc):
    return [word for word in mecab.morphs(doc) if word not in SW and len(word) > 1]

In [ ]:
import konlpy 
from konlpy.tag import Mecab, Kkma, Okt, Komoran
import json
import os
import re
from pprint import pprint

# 형태소 분석기 불러오기
okt = Okt() # 혹시나 mecab 설치 실패한 분들 위해서
mecab = Mecab()

SW = define_stopwords("./stopwords-ko.txt")

for i in range(len(train_df)):
    text = text_cleaning(train_df['comment'][i])
    train_df['comment'][i] = text

###  BoW(Bag of Words) 

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

y_target = train_df['tag']
X_features = train_df.drop('tag', axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, 
                                                    test_size=0.2,
                                                   random_state=156)

### LogisticRegression

In [ ]:
"from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', LogisticRegression())])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

In [ ]:
# text cleaning X

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', LogisticRegression())])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))


### MultinomialNB

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', MultinomialNB())])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

In [ ]:
# text Cleaning X
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', MultinomialNB())])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))


### SGDClassifier

In [ ]:
# 성능 젤 good

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SVC(kernel='linear'))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

In [ ]:
# text Cleaning X
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SVC(kernel='linear'))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

### SVC

In [ ]:
pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SVC(kernel='rbf'))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

In [ ]:
# text Cleaning X
pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SVC(kernel='rbf'))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SVC(kernel='linear'))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

### SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SGDClassifier(loss='perceptron', penalty='l2',
                         alpha=1e-4, random_state=42, 
                         max_iter=500))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

In [ ]:
# text Cleaning X

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', SGDClassifier(loss='perceptron', penalty='l2',
                         alpha=1e-4, random_state=42, 
                         max_iter=500))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

### LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier

pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(tokenizer=text_tokenizing, ngram_range=(1,2), max_df=0.95)),
    ('lr_clf', LGBMClassifier(n_estimators=500,
                     num_leaves=24,
                     boost_from_average=False,
                    is_unbalance = True))])

pipeline.fit(X_train['comment'], y_train)
pred = pipeline.predict(X_test['comment'])

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred, average="macro"),
                                                precision_score(y_test, pred, average="macro"), recall_score(y_test, pred, average="macro")))

## SMOTE

In [ ]:
# SMOTE로 불균형 데이터 처리 후
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelBinarizer

smote = SMOTE(random_state = 150)

tfidf_vect = TfidfVectorizer(tokenizer=text_tokenizing, 
                             ngram_range=(1,2), 
                             max_df=0.95)

tfidf_vect.fit(X_train['comment'])
tfidf_vect_train = tfidf_vect.transform(X_train['comment'])

In [ ]:
X_train_over, y_train_over = smote.fit_sample(tfidf_vect_train, y_train)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())


In [ ]:
clf = SVC(kernel='linear')
clf.fit(X_train_over, y_train_over)

tfidf_vect_test = tfidf_vect.transform(X_test['comment'])
pred_over = clf.predict(tfidf_vect_test)

print('예측 F1_score: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(f1_score(y_test, pred_over, average="macro"),
                                                precision_score(y_test, pred_over, average="macro"), recall_score(y_test, pred_over, average="macro")))

#### Test 데이터 & Submit

In [ ]:
for i in range(len(test_df)):
    text = text_cleaning(test_df['comment'][i])
    test_df['comment'][i] = text

In [ ]:
# train데이터 적용했던 TfidfVectorizer이용하여 test데이터도 피처변환
tfidf_matrix_test_df = tfidf_vect.transform(test_df['comment'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator_df = grid_cv.best_estimator_
preds_test = best_estimator_df.predict(tfidf_matrix_test_df)

#print('Logistic Regression 정확도: ', accuracy_score(y_test, preds))
preds_test[:30]

In [ ]:
# 성능 젤 good

test_pred = pipeline.predict(test_df['comment'])
test_pred

In [ ]:
type(test_pred)
test_df['tag']= test_pred
test_df

In [ ]:
submission = test_df['tag']

In [ ]:
submission.to_csv('./submit_yh2_army.tsv',index=False, header=False, sep="\t")

In [ ]:
from nipa.taskSubmit import nipa_submit

team_id="1345"
task_no="145"
nipa_submit(team_id=team_id,
            task_no=task_no,
            result='./submit_yh2_army.tsv')

### Word2Vec(Word Embedding to Vector)
- BoW기법 -> 고차원의 sparse한 벡터 -> 성능 잘 안나옴
- Word2Vec -> 저차원의 dense vector
    - '주위 단어가 비슷하면 해당 단어의 의미는 유사하다'라는 아이디어에서 시작
    - 거리가 가까울수록 의미가 비슷

In [ ]:
train_df = train_org.copy()

In [ ]:
for i in range(len(train_df)):
    text = text_cleaning(train_df['comment'][i])
    train_df['comment'][i] = text

for i in range(len(train_df)):
    text = text_tokenizing(train_df['comment'][i])
    train_df['comment'][i] = text

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

y_target = train_df['tag']
X_features = train_df.drop('tag', axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, 
                                                    test_size=0.2,
                                                   random_state=156)

In [ ]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(X_train['comment'], 
                           size=100, window=5, 
                           min_count=5, 
                           workers=4, 
                           iter=100, 
                           sg=1)

In [ ]:
# check embedding result
# 비교할 단어와 가장 비슷한(코사인 유사도가 큰) 100개 단어
print(embedding_model.most_similar(positive=["사람"], topn=10))